In [1]:
import ee
import datetime
import os
import itertools
import sys

from pprint import pprint
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [2]:
ee.Initialize()

### Define basic parameters

In [3]:
# define the periods to perform temporal check
Check_len = 3
Check_wieght = [3,1,1]
Check_threshold = 3

# define the number of iteration
Check_iteration_num = 10

In [4]:
# define the names of each period
year_range = list(f'{i[0]}_{i[1]}' for i in zip(range(1990,2018,3),range(1992,2020,3)))

In [5]:
year_range

['1990_1992',
 '1993_1995',
 '1996_1998',
 '1999_2001',
 '2002_2004',
 '2005_2007',
 '2008_2010',
 '2011_2013',
 '2014_2016',
 '2017_2019']

### Import Classified_random_img

In [6]:
# get the classified_random_imgs and sum them up for each period
Classified_Landsat_1990_2019 = [ee.ImageCollection(f"users/wangjinzhulala/North_China_Plain_Python/classification_img/Control_{year}")
                                  .sum().gte(8).set('name',year)   for year in year_range]

Classified_Sentinel_2014_2019 = [ee.ImageCollection(f"users/wangjinzhulala/North_China_Plain_Python/classification_img/Sentinel_Landsat_{year}")
                                  .sum().gte(8).set('name',year)   for year in year_range[-2:]]

# combine classification img together
Classified_imgs = Classified_Landsat_1990_2019[:-2] + Classified_Sentinel_2014_2019

In [26]:
def Temporal_check(mode,in_tifs,weights,threshold):
    
    # get the name of the first img as the property for return img
    name = in_tifs[0].get('name')

    # zip classified_random_sum with weights
    img_multiply = list(zip(in_tifs,weights))

    
    
    if mode == 'forward':
        
        # multiply each classified_sum_img with check_weight 
        # and sum 3-periods together
        sum_tif = ee.ImageCollection([i[0].multiply(i[1]) for i in img_multiply]).sum()
        
        # thoes pixel that GREATER THAN are built-up pixel
        temporal_checked = sum_tif.gt(threshold + 1).set('name',name)
        
    elif mode == 'backward':
                
        #_______________________Get those pixel that was transformed from built to non-built______________
        
        # remap the img,so the pixel value changed (built-->0; non-built -->weight)
                                                                # here need to rename other wise bandname changed to 'remap'
                                                                # which will cause error to add with forward imgs
        backward_remap    = [i[0].remap([0,1],[i[1],0]).rename('classification') for i in img_multiply]
        
        # sum the back_remap and the those pixels with value GREATE THAN the threshold is "Non-Built"
        non_built = ee.ImageCollection(backward_remap).sum().gt(threshold + 1)
        
        # change the pixel value to get the right pixel value (0--> non-built; 1-->built)
        temporal_checked = non_built.Not()
        
        
    else:
        print("Please provide a correct mode ['forward'|'backward']")
    
    return temporal_checked

In [19]:
def Forward_backward(temporal_check_len,in_imgs):
    
    img_idx_for_temporal_check = range(len(in_imgs) - temporal_check_len + 1)
    
    #_______________________________Forward temporal check________________________________________________
    
    forward_chunks  = [in_imgs[i:i + temporal_check_len] for i in img_idx_for_temporal_check]
    # perform the forward temporal check
    forward_tif     = [Temporal_check('forward',chunk,Check_wieght,Check_threshold) for chunk in forward_chunks]
    
    
    
    #_______________________________Backward temporal check________________________________________________
    
    # combie the forward_checked tifs with unchecked imgs in the tail, and reverse the list order
    backward_tifs = forward_tif + Classified_imgs[len(forward_tif):]
    backward_tifs_reverse = backward_tifs[::-1]
 
    # slice the backward_tifs into chunks with the length of temporal_check_len
    backward_chunks = [backward_tifs_reverse[i:i + temporal_check_len] for i in img_idx_for_temporal_check]
    
    # perform the backward temporal check
    backward_tif    = [Temporal_check('backward',chunk,Check_wieght,Check_threshold) for chunk in backward_chunks]

    
    
    #___________________________Add Forward & Backward checked img together___________________________________
    
    # reorder the backward_tif so tifs to ensure an ascending time order
    re_order_tif = backward_tif[::-1]
    
    forward_backward = forward_tif[:temporal_check_len-1] + re_order_tif
    
    return forward_backward

In [29]:
def Forward_backward(temporal_check_len,in_imgs):
    
    img_idx_for_temporal_check = range(len(in_imgs) - temporal_check_len + 1)
    
    #_______________________________Forward temporal check________________________________________________
    
    forward_chunks  = [in_imgs[i:i + temporal_check_len] for i in img_idx_for_temporal_check]
    # perform the forward temporal check
    forward_tif     = [Temporal_check('forward',chunk,Check_wieght,Check_threshold) for chunk in forward_chunks]

    
    
    #_______________________________Backward temporal check________________________________________________
    
    # combie the forward_checked tifs with unchecked imgs in the tail, and reverse the list order
    backward_tifs = forward_tif + Classified_imgs[len(forward_tif):]
    backward_tifs_reverse = backward_tifs[::-1]
 
    # slice the backward_tifs into chunks with the length of temporal_check_len
    backward_chunks = [backward_tifs_reverse[i:i + temporal_check_len] for i in img_idx_for_temporal_check]
    
    # perform the backward temporal check
    backward_tif    = [Temporal_check('backward',chunk,Check_wieght,Check_threshold) for chunk in backward_chunks]

    
    
    #___________________________Add Forward & Backward checked img together___________________________________
    
    # reorder the backward_tif so tifs to ensure an ascending time order
    re_order_tif = backward_tif[::-1]
    
    forward_backward = forward_tif[:temporal_check_len-1] + re_order_tif
    
    return forward_backward

In [30]:
Iter_temporal_check_instaces = {}

for it in range(Check_iteration_num):

    if it == 0:
        
        in_imgs = Classified_imgs
        
        forward_backward_checked =  Forward_backward(Check_len,in_imgs)
        forward_backward_with_iteration = [ee.Image(img).set('iteration',it) for img in forward_backward_checked]
        
        Iter_temporal_check_instaces[0]  = forward_backward_with_iteration
        
        
    else:
        
        in_imgs = Iter_temporal_check_instaces[it-1]
        
        forward_backward_checked =  Forward_backward(Check_len,in_imgs)
        forward_backward_with_iteration = [ee.Image(img).set('iteration',it) for img in forward_backward_checked]
        
        Iter_temporal_check_instaces[it]  = forward_backward_with_iteration

In [31]:
Iter_temporal_check_instaces

{0: [<ee.image.Image at 0x22233080320>,
 1: [<ee.image.Image at 0x22233088160>,
 2: [<ee.image.Image at 0x2223309e2e8>,
 3: [<ee.image.Image at 0x222330b73c8>,
 4: [<ee.image.Image at 0x222330d1470>,
 5: [<ee.image.Image at 0x222330e8550>,
 6: [<ee.image.Image at 0x22233102630>,
 7: [<ee.image.Image at 0x2223311d710>,
 8: [<ee.image.Image at 0x222331347f0>,
 9: [<ee.image.Image at 0x2223314f8d0>,
  <ee.image.Image at 0x22233160080>]}

In [34]:
Map = geemap.Map()
Map.setCenter(115.4508, 35.2492,10)

Map.add_basemap('HYBRID')
Map.addLayer(Classified_imgs[8] ,{'min':0,'max':1},'origin')
Map.addLayer(Iter_temporal_check_instaces[0][8] ,{'min':0,'max':1},'Iter_0')
Map.addLayer(Iter_temporal_check_instaces[1][8] ,{'min':0,'max':1},'Iter_1')
Map.addLayer(Iter_temporal_check_instaces[2][8] ,{'min':0,'max':1},'Iter_2')
Map.addLayer(Iter_temporal_check_instaces[3][8] ,{'min':0,'max':1},'Iter_3')
Map.addLayer(Iter_temporal_check_instaces[4][8] ,{'min':0,'max':1},'Iter_4')
Map.addLayer(Iter_temporal_check_instaces[5][8] ,{'min':0,'max':1},'Iter_5')
Map.addLayer(Iter_temporal_check_instaces[6][8] ,{'min':0,'max':1},'Iter_6')
Map.addLayer(Iter_temporal_check_instaces[7][8] ,{'min':0,'max':1},'Iter_7')
Map.addLayer(Iter_temporal_check_instaces[8][8] ,{'min':0,'max':1},'Iter_8')
Map.addLayer(Iter_temporal_check_instaces[9][8] ,{'min':0,'max':1},'Iter_9')

Map

Map(center=[35.2492, 115.4508], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButt…

In [34]:
table = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain_Python/Boundary_shp/North_China_Plain_Boundary")

In [36]:
freq = Iter_temporal_check_instaces[0][0].reduceRegion(ee.Reducer.frequencyHistogram(),table.geometry(),30,bestEffort=True)

In [37]:
freq.getInfo()

{'classification': {'0': 9194238.980392165, '1': 741167.7686274509}}